<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosPrevisaoProximaSentencaBERT_pt_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Previsão Próxima Sentença usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos de uso de **Word Embeddings Contextuais** para previsão da próxima sentença.


**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

## 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

###Tratamento de logs

Método para tratamento dos logs.

In [ ]:
#biblioteca de logging
import logging

#formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
#se estiver executando no Google Colaboratory
import sys
#retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1 - Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [ ]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.5.1

## 2 - Download do arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [ ]:
# Importando as bibliotecas
import os

# Variável para setar o arquivo
URL_MODELO = None

# Comente uma das urls para carregar modelos de tamanhos diferentes(base/large)
# URL_MODELO do arquivo do modelo tensorflow
# arquivo menor(base) 1.1 Gbytes
#URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip"

# arquivo grande(large) 3.5 Gbytes
URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip"

# Se a variável foi setada
if URL_MODELO:

    # Diretório descompactação
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da URL_MODELO
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na URL_MODELO
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe na pasta corrente
    if os.path.exists(DIRETORIO_MODELO):
      print("Apagando diretório existente do modelo!")
      # Apaga a pasta e os arquivos existentes
      !rm -rf $DIRETORIO_MODELO    

    # Baixa o arquivo do modelo
    !wget $URL_MODELO
    
    # Descompacta o arquivo na pasta de descompactação
    !unzip -o $arquivo -d $DIRETORIO_MODELO

    # Baixa o arquivo do vocabulário
    # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente
    URL_MODELO_VOCAB = caminho + arquivo_vocab
    !wget $URL_MODELO_VOCAB
    
    # Coloca o arquivo do vocabulário no diretório de descompactação
    !mv $arquivo_vocab $DIRETORIO_MODELO
            
    # Move o arquivo para pasta de descompactação
    !mv $arquivo $DIRETORIO_MODELO
       
    print('Pasta do ' + DIRETORIO_MODELO + ' pronta!')
    
    # Lista a pasta corrente
    !ls -la $DIRETORIO_MODELO
else:
    print('Variável URL_MODELO não setada!')

Apagando diretório existente do modelo!
--2021-08-16 12:02:53--  https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip
Resolving neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)... 52.219.105.178
Connecting to neuralmind-ai.s3.us-east-2.amazonaws.com (neuralmind-ai.s3.us-east-2.amazonaws.com)|52.219.105.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1244275810 (1.2G) [application/zip]
Saving to: ‘bert-large-portuguese-cased_pytorch_checkpoint.zip’

bert-large-portugue 100%[===================>]   1.16G  93.6MB/s    in 13s     

2021-08-16 12:03:06 (92.3 MB/s) - ‘bert-large-portuguese-cased_pytorch_checkpoint.zip’ saved [1244275810/1244275810]

Archive:  bert-large-portuguese-cased_pytorch_checkpoint.zip
  inflating: /content/modelo/config.json  
  inflating: /content/modelo/pytorch_model.bin  
--2021-08-16 12:03:27--  https://neuralmind-ai.s3.u

## 3 - Carregando o Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

In [ ]:
# Importando as bibliotecas do tokenizador
from transformers import BertTokenizer

tokenizer = None

# Se a variável URL_MODELO foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carrgando o tokenizador BERT do diretório ' + DIRETORIO_MODELO + '...')

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=False)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o tokenizador da comunidade...')
    
    #tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)

Carrgando o tokenizador BERT do diretório /content/modelo...


## 4 - Carregando o Modelo BERT(BertModel)

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BertForNextSentencePrediction

# Se a variável URL_MODELO1 foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carregando o modelo BERT do diretório ' + DIRETORIO_MODELO + '...')

    model = BertForNextSentencePrediction.from_pretrained(DIRETORIO_MODELO, 
                                      output_hidden_states = True)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o modelo BERT da comunidade ...')

    model = BertForNextSentencePrediction.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                       output_hidden_states = True)
    

Carregando o modelo BERT do diretório /content/modelo...


Some weights of the model checkpoint at /content/modelo were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.eval()

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

## 5 - Função de comparação

In [ ]:
import torch

def pontuacaoSentenca(entrada, candidato):			
  # pode ser melhor concatenar * após * a tokenização usando token especial [SEP]
  combinado = entrada + ' [SEP] ' + candidato
  entrada_tokenizada = tokenizer.encode(combinado, add_special_tokens=True)
	# Gera a entrada como tokenizada e o unsqueeze insere uma dimensão na posição 0
  entrada_tokenizada_indexada = torch.tensor(entrada_tokenizada).unsqueeze(0)
  #print(entrada_tokenizada_indexada)
	# Submete o texto ao modela
  outputs = model(entrada_tokenizada_indexada)  
	# outputs é uma matriz com perdas como o primeiro valor e logits como o segundo
  # outputs[0] = last_hidden_state
  # outputs[0][0] é igual outputs.logits[0]. Logits retorna dois valores, referenciado pelo terceiro índice.      
  # A função `.item()` retorna apenas o valor Python do tensor.       
  print("Saída:",outputs[0][0])
  pontuacao = outputs.logits[0][0].item()   
  #print(candidato, ":", pontuacao)
  return pontuacao

## 6 - Exemplos

### Comparando sentenças

In [ ]:
sentenca1 = "Hoje está calor."
sentenca2 = "Vamos a praia."
sentenca3 = "O carro estragou."
sentenca4 = "Vou levar a oficina."

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado = pontuacaoSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado = pontuacaoSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado = pontuacaoSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado = pontuacaoSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)


Sentença 1: Hoje está calor.
Sentença 2: Vamos a praia.
Saída: tensor([ 0.6251, -0.9251], grad_fn=<SelectBackward>)
Resultado da relação da sentença 2: 0.625061571598053

Sentença 1: Hoje está calor.
Sentença 3: O carro estragou.
Saída: tensor([0.1169, 0.5731], grad_fn=<SelectBackward>)
Resultado da relação da sentença 3: 0.11686892062425613

Sentença 3: O carro estragou.
Sentença 4: Vou levar a oficina.
Saída: tensor([ 0.9069, -1.0739], grad_fn=<SelectBackward>)
Resultado da relação da sentença 4: 0.9069446325302124

Sentença 3: O carro estragou.
Sentença 1: Hoje está calor.
Saída: tensor([ 0.2409, -0.0578], grad_fn=<SelectBackward>)
Resultado da relação da sentença 1: 0.240912064909935


### Comparando uma sequência de sentenças 1

In [ ]:
# Define um texto
texto = ['Fui de carro.',
         'O caminhão tombou.',              
         'O ônibus atrasou.']

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))

Saída: tensor([ 0.8284, -0.1576], grad_fn=<SelectBackward>)
Fui de carro.  =>  O caminhão tombou. : 0.8284463286399841
Saída: tensor([ 0.5831, -0.0621], grad_fn=<SelectBackward>)
O caminhão tombou.  =>  O ônibus atrasou. : 0.5830506682395935
Total sentenças: 3
Total: 1.4114969968795776
Média: 0.4704989989598592


### Comparando uma sequência de sentenças 2

In [ ]:
# Define um texto com a permutação das sentenças do texto original
texto = ['O gato saiu correndo.',   
             'O cachorro latiu.',             
             'O passáro está no ninho.']    

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Saída: tensor([0.1706, 0.7546], grad_fn=<SelectBackward>)
O gato saiu correndo.  =>  O cachorro latiu. : 0.1705625355243683
Saída: tensor([-0.0530, -0.1603], grad_fn=<SelectBackward>)
O cachorro latiu.  =>  O passáro está no ninho. : -0.05296250060200691
Total sentenças: 3
Total: 0.2235250361263752
Média: 0.0745083453754584


### Comparando texto original com permutado

In [ ]:
print("Texto Original")

# Define um texto com 4 sentenças
texto_original = ['Bom Dia, professor.',
             'Qual o conteúdo da prova?',              
             'Vai cair tudo na prova?',
             'Aguardo uma resposta, João.']

texto = texto_original

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + pontuacao
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

print("\nTexto permutado")
# Define um texto com a permutação das sentenças do texto original
texto = [texto_original[3],   # 'Aguardo uma resposta, João.',
             texto_original[1],             # 'Qual o conteúdo da prova?',              
             texto_original[0],             # 'Vai cair tudo na prova?',
             texto_original[2]]             # 'Bom Dia, professor.']                 

total = 0
for i in range(len(texto)-1):
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Texto Original
Saída: tensor([ 1.5311, -0.2564], grad_fn=<SelectBackward>)
Bom Dia, professor.  =>  Qual o conteúdo da prova? : 1.5311131477355957
Saída: tensor([0.8600, 0.5956], grad_fn=<SelectBackward>)
Qual o conteúdo da prova?  =>  Vai cair tudo na prova? : 0.860034167766571
Saída: tensor([ 1.4897, -1.0947], grad_fn=<SelectBackward>)
Vai cair tudo na prova?  =>  Aguardo uma resposta, João. : 1.4897127151489258
Total sentenças: 4
Total: 3.8808600306510925
Média: 0.9702150076627731

Texto permutado
Saída: tensor([ 1.5511, -0.4897], grad_fn=<SelectBackward>)
Aguardo uma resposta, João.  =>  Qual o conteúdo da prova? : 1.5511293411254883
Saída: tensor([ 1.0969, -0.0119], grad_fn=<SelectBackward>)
Qual o conteúdo da prova?  =>  Bom Dia, professor. : 1.0969069004058838
Saída: tensor([1.2152, 0.1825], grad_fn=<SelectBackward>)
Bom Dia, professor.  =>  Vai cair tudo na prova? : 1.2151997089385986
Total sentenças: 4
Total: 3.8632359504699707
Média: 0.9658089876174927


### Comparando palavras

In [ ]:
sentenca1 = "cachorro"
sentenca2 = "gato"
sentenca3 = "avião"
sentenca4 = "ônibus"

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado = pontuacaoSentenca(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado = pontuacaoSentenca(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado = pontuacaoSentenca(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado = pontuacaoSentenca(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)


Sentença 1: cachorro
Sentença 2: gato
Saída: tensor([-0.9021,  0.8146], grad_fn=<SelectBackward>)
Resultado da relação da sentença 2: -0.9021257758140564

Sentença 1: cachorro
Sentença 3: avião
Saída: tensor([-0.3603,  0.9946], grad_fn=<SelectBackward>)
Resultado da relação da sentença 3: -0.3602618873119354

Sentença 3: avião
Sentença 4: ônibus
Saída: tensor([0.3336, 0.0673], grad_fn=<SelectBackward>)
Resultado da relação da sentença 4: 0.3336144983768463

Sentença 3: avião
Sentença 1: cachorro
Saída: tensor([-0.4827,  0.9095], grad_fn=<SelectBackward>)
Resultado da relação da sentença 1: -0.48266738653182983


### Grupo de palavras relacionadas

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'cavalo',
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
Saída: tensor([-0.9021,  0.8146], grad_fn=<SelectBackward>)
cachorro  =>  gato : -0.9021257758140564
Saída: tensor([-0.0915,  1.0238], grad_fn=<SelectBackward>)
gato  =>  cavalo : -0.0914822444319725
Saída: tensor([0.0490, 1.0001], grad_fn=<SelectBackward>)
cavalo  =>  zebra : 0.04902910441160202
Total sentenças: 4
Total: 1.042637124657631
Média: 0.26065928116440773


### Grupo de palavras relacionadas com ruído

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'avião', #<= Ruído
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao = pontuacaoSentenca(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
Saída: tensor([-0.9021,  0.8146], grad_fn=<SelectBackward>)
cachorro  =>  gato : -0.9021257758140564
Saída: tensor([0.1469, 1.3691], grad_fn=<SelectBackward>)
gato  =>  avião : 0.14692886173725128
Saída: tensor([0.2148, 0.8442], grad_fn=<SelectBackward>)
avião  =>  zebra : 0.21484261751174927
Total sentenças: 4
Total: 1.263897255063057
Média: 0.31597431376576424


## 7 - Função de comparação com softmax

In [ ]:
import torch
from torch.nn.functional import softmax

def pontuacaoSentenca2(entrada, candidato):			  
  entrada_tokenizada_indexada = tokenizer.encode_plus(entrada, text_pair=candidato, return_tensors='pt')
	# Submete o texto ao modelo
  #outputs = model(entrada_tokenizada_indexada)  
  outputs = model(**entrada_tokenizada_indexada)  
	# outputs é uma matriz com perdas como o primeiro valor e logits como o segundo
  # outputs[0] = last_hidden_state
  # outputs[0][0] é igual outputs.logits[0]. Logits retorna dois valores, referenciado pelo terceiro índice.      
  # A função `.item()` retorna apenas o valor Python do tensor.       
  #print("Saída:",outputs[0][0])
  #Recupera a probabilidade da sequência
  probs = softmax(outputs[0], dim=1)
  # valor muito alto para o índice 0: alta probabilidade de candidato ser uma continuação da entrada
  #print("Probabilidade:",probs)
  pontuacao = outputs.logits[0][0].item()   
  #print(candidato, ":", pontuacao)
  return pontuacao, probs

### Comparando sentenças

In [ ]:
sentenca1 = "Hoje está calor."
sentenca2 = "Vamos a praia."
sentenca3 = "O carro estragou."
sentenca4 = "Vou levar a oficina."

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado, probabilidade = pontuacaoSentenca2(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado, probabilidade = pontuacaoSentenca2(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado, probabilidade = pontuacaoSentenca2(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado, probabilidade = pontuacaoSentenca2(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)
print("Probabilidade:", probabilidade)


Sentença 1: Hoje está calor.
Sentença 2: Vamos a praia.
Resultado da relação da sentença 2: 2.4943301677703857
Probabilidade: tensor([[0.9903, 0.0097]], grad_fn=<SoftmaxBackward>)

Sentença 1: Hoje está calor.
Sentença 3: O carro estragou.
Resultado da relação da sentença 3: 0.8492786884307861
Probabilidade: tensor([[0.9167, 0.0833]], grad_fn=<SoftmaxBackward>)

Sentença 3: O carro estragou.
Sentença 4: Vou levar a oficina.
Resultado da relação da sentença 4: 3.924736499786377
Probabilidade: tensor([[0.9947, 0.0053]], grad_fn=<SoftmaxBackward>)

Sentença 3: O carro estragou.
Sentença 1: Hoje está calor.
Resultado da relação da sentença 1: 0.8521135449409485
Probabilidade: tensor([[0.9088, 0.0912]], grad_fn=<SoftmaxBackward>)


### Comparando uma sequência de sentenças 1

In [ ]:
# Define um texto
texto = ['Fui de carro.',
         'O caminhão tombou.',              
         'O ônibus atrasou.']

total = 0
for i in range(len(texto)-1):
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))

Fui de carro.  =>  O caminhão tombou. : 1.0553452968597412
    Probabilidade: tensor([[0.9119, 0.0881]], grad_fn=<SoftmaxBackward>)
O caminhão tombou.  =>  O ônibus atrasou. : 9.274107933044434
    Probabilidade: tensor([[9.9913e-01, 8.7166e-04]], grad_fn=<SoftmaxBackward>)
Total sentenças: 3
Total: 10.329453229904175
Média: 3.443151076634725


### Comparando uma sequência de sentenças 2

In [ ]:
# Define um texto com a permutação das sentenças do texto original
texto = ['O gato saiu correndo.',   
             'O cachorro latiu.',             
             'O passáro está no ninho.']    

total = 0
for i in range(len(texto)-1):
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

O gato saiu correndo.  =>  O cachorro latiu. : 13.274903297424316
    Probabilidade: tensor([[9.9991e-01, 8.9398e-05]], grad_fn=<SoftmaxBackward>)
O cachorro latiu.  =>  O passáro está no ninho. : 3.625040054321289
    Probabilidade: tensor([[0.9946, 0.0054]], grad_fn=<SoftmaxBackward>)
Total sentenças: 3
Total: 16.899943351745605
Média: 5.633314450581868


### Comparando texto original com permutado

In [ ]:
print("Texto Original")

# Define um texto com 4 sentenças
texto_original = ['Bom Dia, professor.',
             'Qual o conteúdo da prova?',              
             'Vai cair tudo na prova?',
             'Aguardo uma resposta, João.']

texto = texto_original

total = 0
for i in range(len(texto)-1):    
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

print("\nTexto permutado")
# Define um texto com a permutação das sentenças do texto original
texto = [texto_original[3],   # 'Aguardo uma resposta, João.',
             texto_original[1],             # 'Qual o conteúdo da prova?',              
             texto_original[0],             # 'Vai cair tudo na prova?',
             texto_original[2]]             # 'Bom Dia, professor.']                 

total = 0
for i in range(len(texto)-1):
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Texto Original
Bom Dia, professor.  =>  Qual o conteúdo da prova? : 2.0516791343688965
    Probabilidade: tensor([[0.9534, 0.0466]], grad_fn=<SoftmaxBackward>)
Qual o conteúdo da prova?  =>  Vai cair tudo na prova? : 6.653902053833008
    Probabilidade: tensor([[0.9985, 0.0015]], grad_fn=<SoftmaxBackward>)
Vai cair tudo na prova?  =>  Aguardo uma resposta, João. : 2.2361972332000732
    Probabilidade: tensor([[0.9813, 0.0187]], grad_fn=<SoftmaxBackward>)
Total sentenças: 4
Total: 10.941778421401978
Média: 2.7354446053504944

Texto permutado
Aguardo uma resposta, João.  =>  Qual o conteúdo da prova? : 0.10518847405910492
    Probabilidade: tensor([[0.2178, 0.7822]], grad_fn=<SoftmaxBackward>)
Qual o conteúdo da prova?  =>  Bom Dia, professor. : 1.3705203533172607
    Probabilidade: tensor([[0.9487, 0.0513]], grad_fn=<SoftmaxBackward>)
Bom Dia, professor.  =>  Vai cair tudo na prova? : 1.2606265544891357
    Probabilidade: tensor([[0.8051, 0.1949]], grad_fn=<SoftmaxBackward>)
Total sente

### Comparando palavras

In [ ]:
sentenca1 = "cachorro"
sentenca2 = "gato"
sentenca3 = "avião"
sentenca4 = "ônibus"

print("Sentença 1:", sentenca1)
print("Sentença 2:", sentenca2)
resultado, probabilidade = pontuacaoSentenca2(sentenca1, sentenca2)
print("Resultado da relação da sentença 2:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 1:", sentenca1)
print("Sentença 3:", sentenca3)
resultado, probabilidade = pontuacaoSentenca2(sentenca1, sentenca3)
print("Resultado da relação da sentença 3:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 3:", sentenca3)
print("Sentença 4:", sentenca4)
resultado, probabilidade = pontuacaoSentenca2(sentenca3, sentenca4)
print("Resultado da relação da sentença 4:", resultado)
print("Probabilidade:", probabilidade)

print("\nSentença 3:", sentenca3)
print("Sentença 1:", sentenca1)
resultado, probabilidade = pontuacaoSentenca2(sentenca3, sentenca1)
print("Resultado da relação da sentença 1:", resultado)
print("Probabilidade:", probabilidade)

Sentença 1: cachorro
Sentença 2: gato
Resultado da relação da sentença 2: 13.916265487670898
Probabilidade: tensor([[9.9992e-01, 8.0443e-05]], grad_fn=<SoftmaxBackward>)

Sentença 1: cachorro
Sentença 3: avião
Resultado da relação da sentença 3: 6.6345343589782715
Probabilidade: tensor([[0.9980, 0.0020]], grad_fn=<SoftmaxBackward>)

Sentença 3: avião
Sentença 4: ônibus
Resultado da relação da sentença 4: 6.271944999694824
Probabilidade: tensor([[0.9977, 0.0023]], grad_fn=<SoftmaxBackward>)

Sentença 3: avião
Sentença 1: cachorro
Resultado da relação da sentença 1: 4.632396221160889
Probabilidade: tensor([[0.9963, 0.0037]], grad_fn=<SoftmaxBackward>)


### Grupo de palavras relacionadas

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'cavalo',
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
cachorro  =>  gato : 13.916265487670898
    Probabilidade: tensor([[9.9992e-01, 8.0443e-05]], grad_fn=<SoftmaxBackward>)
gato  =>  cavalo : 10.360709190368652
    Probabilidade: tensor([[9.9962e-01, 3.8360e-04]], grad_fn=<SoftmaxBackward>)
cavalo  =>  zebra : 3.6907591819763184
    Probabilidade: tensor([[0.9909, 0.0091]], grad_fn=<SoftmaxBackward>)
Total sentenças: 4
Total: 27.96773386001587
Média: 6.991933465003967


### Grupo de palavras relacionadas com ruído

In [ ]:
print("Grupo palavras")

# Define uma lista com 4 palavras
texto = ['cachorro',
         'gato',              
         'avião', #<= Ruído
         'zebra']

total = 0
for i in range(len(texto)-1):    
    pontuacao, probabilidade = pontuacaoSentenca2(texto[i], texto[i+1])
    total = total + abs(pontuacao)
    print(texto[i], " => ",texto[i+1], ":", pontuacao)
    print("    Probabilidade:", probabilidade)

print("Total sentenças:", len(texto))
print("Total:",total)
print("Média:",total/len(texto))           

Grupo palavras
cachorro  =>  gato : 13.916265487670898
    Probabilidade: tensor([[9.9992e-01, 8.0443e-05]], grad_fn=<SoftmaxBackward>)
gato  =>  avião : 6.556914329528809
    Probabilidade: tensor([[0.9975, 0.0025]], grad_fn=<SoftmaxBackward>)
avião  =>  zebra : 1.603911280632019
    Probabilidade: tensor([[0.9563, 0.0437]], grad_fn=<SoftmaxBackward>)
Total sentenças: 4
Total: 22.077091097831726
Média: 5.5192727744579315
